# ISD - Travail pratique 05 – Régression linéaire
## Objectifs:
- Appliquer l’algorithme de régression linéaire simple pour créer des modèles de régression
- Utiliser la méthode de validation hold-out pour évaluer un modèle de régression
- Utiliser les bibliothèques d’apprentissage automatique pour construire des modèles de régression multiple
- Évaluer les modèles et analyser les résultats